# HW2 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  2
* **Date:**  5/23/16

**HW2.1.** Sort in Hadoop MapReduce.  **Given as input:** Records of the form (integer, “NA”), where integer is any integer, and “NA” is just the empty string.  **Output:** sorted key value pairs of the form (integer, “NA”) in decreasing order; what happens if you have multiple reducers? Do you need additional steps? Explain.

**Part 1** Write code to generate N  random records of the form (integer, “NA”). Let N = 10,000.

**Part 2** Write the python Hadoop streaming map-reduce job to perform this sort. Display the top 10 biggest numbers. Display the 10 smallest numbers

In [2]:
import random

N=10000
min_int = -100000
max_int = 100000
filename = "random_integers.txt"
with open(filename, 'w') as f:
    for i in range(N):
        f.write('<%d, "">\n' % (random.randint(min_int, max_int)))

In [9]:
# Create a directory in HDFS to store the data for this problem
#!/usr/local/hadoop/bin/hdfs dfs -mkdir /user
#!/usr/local/hadoop/bin/hdfs dfs -mkdir /user/hadoop

16/05/20 22:10:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
# Read the input file into HDFS
#!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal random_integers.txt /user/hadoop

16/05/20 22:11:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
drwxr-xr-x   - hadoop supergroup          0 2016-05-20 22:10 /user/hadoop/outputHW2-1
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [22]:
# Check to make sure the file is there.
#!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop

16/05/20 22:33:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 1 items
-rw-r--r--   3 hadoop supergroup     123850 2016-05-20 22:09 /user/hadoop/random_integers.txt


In [17]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW2.1.

import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove the angle brackets
    line = line.strip('<>\n')
    # split the line into words
    words = line.split(',')
    # write the results to STDOUT (standard output);
    # what we output here will be the input for the
    # Reduce step, i.e. the input for reducer.py
    print('%s' % (words[0]))

Overwriting mapper.py


In [43]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW2.1.

import sys

N=10000
i=0
# input comes from STDIN
for line in sys.stdin:
    if (i>=0 and i<10) or (i>=(N-10) and i<N):
        print('<%s, "">' % (line.strip()))
    i += 1

Overwriting reducer.py


In [44]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options=-nr \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/random_integers.txt -output /user/hadoop/outputHW2-1

16/05/21 14:00:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/outputHW2-1/*': No such file or directory
16/05/21 14:00:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:00:50 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar4921037629859414604/] [] /tmp/streamjob3968137248463036228.jar tmpDir=null
16/05/21 14:00:50 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 14:00:50 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/21 14:00:51 INFO mapred.FileInputFormat: Total input paths to process : 1
16/05/21 14:00:51 INFO mapreduce.JobSubmitter: number of splits:2
16/05/21 14:00:51 INFO Configuration.deprecati

In [45]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-1/part-00000

16/05/21 14:01:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 14:01 /user/hadoop/outputHW2-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup        270 2016-05-21 14:01 /user/hadoop/outputHW2-1/part-00000
16/05/21 14:01:23 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
<99999, "">	
<99995, "">	
<99982, "">	
<99974, "">	
<99969, "">	
<99942, "">	
<99929, "">	
<99892, "">	
<99887, "">	
<99882, "">	
<-99775, "">	
<-99842, "">	
<-99867, "">	
<-99889, "">	
<-99912, "">	
<-99939, "">	
<-99943, "">	
<-99961, "">	
<-99984, "">	
<-99994, "">	


In [46]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW2.2.

# Import sys library to access arguments passed in when the module is called
import sys

# input comes from STDIN (standard input)
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip('<>\n')
    # split the line into words
    words = line.split()
    # increase counters
    for word in words:
        # write the results to STDOUT (standard output);
        # what we output here will be the input for the
        # Reduce step, i.e. the input for reducer.py
        #
        # tab-delimited; the trivial word count is 1
        print '%s\t%s' % (word, 1)

Overwriting mapper.py


In [47]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW2.2

from operator import itemgetter
# Import sys library to access arguments passed in when the module is called
import sys

current_word = None
current_count = 0
word = None

# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()

    # parse the input we got from mapper.py
    word, count = line.split('\t', 1)

    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        # count was not a number, so silently
        # ignore/discard this line
        continue

    # this IF-switch only works because Hadoop sorts map output
    # by key (here: word) before it is passed to the reducer
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word

# do not forget to output the last word if needed!
if current_word == word:
    print '%s\t%s' % (current_word, current_count)

Overwriting reducer.py


In [48]:
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/random_integers.txt -output /user/hadoop/outputHW2-1

16/05/21 14:06:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:06:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-1/_SUCCESS
16/05/21 14:06:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW2-1/part-00000
16/05/21 14:06:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/21 14:06:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar7496711587575795057/] [] /tmp/streamjob8403600677380589246.jar tmpDir=null
16/05/21 14:06:23 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205

In [49]:
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW2-1
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW2-1/part-00000

16/05/21 14:06:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-21 14:06 /user/hadoop/outputHW2-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup      91627 2016-05-21 14:06 /user/hadoop/outputHW2-1/part-00000
16/05/21 14:06:47 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
""	10000
-10013,	1
-10041,	1
-10059,	1
-10068,	2
-10090,	1
-10095,	1
-10116,	1
-10132,	1
-10164,	1
-10181,	1
-10227,	1
-10235,	1
-10246,	1
-10247,	1
-1026,	1
-10260,	1
-10273,	1
-10287,	1
-10306,	2
-10307,	1
-10333,	1
-10341,	1
-10348,	1
-10392,	1
-10405,	1
-10414,	1
-10421,	1
-10436,	1
-10454,	1
-1046,	1
-10465,	1
-10493,	1
-10497,	1
-10518,	1
-10545,	1
-10568,	1
-10585,	1
-10611,	1
-10660,	1
-10671,	1
-10686,	1
-10691,	1
-10710,	1
-10720,	1
-10766,	1
-10774,	1
-10781,	1
-10806,	1
-10822,	1
-

In [16]:
print('<-10090, "">\n'.strip('<>\n'))

-10090, ""
